# Create a Slice and Run iPerf3



## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  


In [ ]:
slice_name = 'iPerf3'
[site1, site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name='Node1'
node2_name='Node2'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, cores=4, ram=16, site=site1, image='docker_rocky_8')
node1.add_fabnet()
node1.add_post_boot_upload_directory('node_tools','.')
node1.add_post_boot_execute('node_tools/host_tune.sh')
node1.add_post_boot_execute('node_tools/enable_docker.sh docker_rocky_8 ')
node1.add_post_boot_execute('docker pull pruth/fabric-multitool-rockylinux8:latest ')


# Node2
node2 = slice.add_node(name=node2_name, cores=4, ram=16, site=site2, image='docker_rocky_8')
node2.add_fabnet()
node2.add_post_boot_upload_directory('node_tools','.')
node2.add_post_boot_execute('node_tools/host_tune.sh')
node2.add_post_boot_execute('node_tools/enable_docker.sh docker_rocky_8 ')
node2.add_post_boot_execute('docker pull pruth/fabric-multitool-rockylinux8:latest ')

#Submit Slice Request
slice.submit();

## Run iPerf3

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

stdout1, stderr1 = node1.execute("docker run -d --rm "
                                "--network host "
                                "pruth/fabric-multitool-rockylinux8:latest "
                                "iperf3 -s -1"
                                , quiet=True, output_file=f"{node1.get_name()}.log");

node1_addr = node1.get_interface(network_name=f'FABNET_IPv4_{node1.get_site()}').get_ip_addr()

stdout2, stderr2 = node2.execute("docker run --rm "
                                "--network host "
                                "pruth/fabric-multitool-rockylinux8:latest "
                                f"iperf3 -c {node1_addr} -P 4 -t 30 -i 10 -O 10"
                                , quiet=False, output_file=f"{node2.get_name()}.log");




## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()